In [1]:
!pip install pandas
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [69]:
import pandas as pd
import re
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()
newrules = read_file('newrules.txt')
components = read_file('component.txt')
rules = read_file('rules.txt') 
type_and_etc = [line.split('\t') for line in read_file('type and etc.txt')]
sections = []
buffer = []
with open('OD rules.txt', 'r', encoding='utf-8') as file:
    for line in file:
        stripped_line = line.strip()
        if stripped_line == '':
            if buffer:
                sections.append(' '.join(buffer))
                buffer = []
        else:
            buffer.append(stripped_line)
    if buffer:
        sections.append(' '.join(buffer))
pattern = r'([✔✘])\s*(.*)'
data = []
section_id = 1
for section in sections:
    items = re.split(r'\s*\|\|\s*|\s*&&\s*|\s*\(\s*|\s*\)\s*', section)
    section_data = []
    for item in items:
        match = re.match(pattern, item) 
        if match:
            status, name = match.groups()
            section_data.append((section_id, status, name))
    if section_data:
        data.extend(section_data)
    section_id += 1
df = pd.DataFrame(data, columns=['Section_id', 'Status', 'name'])
df['name'] = df['name'].str.replace(r'\bOnly\b.*', '', regex=True).str.strip()
df['name'] = df['name'].str.replace(r'\bSBB\w*\b', '', regex=True).str.strip()
print(df) 

      Section_id Status                                               name
0              1      ✔      Intel i3-12100 3.3GHz/4C/8T/12M 60W DDR5 4800
1              2      ✔  Intel Pentium Gold G7400 3.7GHz/2C/4T/6M 46W D...
2              3      ✔  Intel Celeron G6900 3.4GHz/2C/2T/4M 46W DDR5 4800
3              4      ✔    Intel i9-12900 2.4GHz/16C/24T/30M 65W DDR5 4800
4              5      ✔    Intel i7-12700 2.1GHz/12C/20T/25M 65W DDR5 4800
...          ...    ...                                                ...
2028         996      ✔                       ThinkSmart IP Controller Kit
2029         997      ✔                       ThinkSmart IP Controller Kit
2030         998      ✔                       ThinkSmart IP Controller Kit
2031         999      ✔                       ThinkSmart IP Controller Kit
2032        1000      ✔                       ThinkSmart IP Controller Kit

[2033 rows x 3 columns]


In [83]:
import sqlite3 
from py2neo import Graph, Node, Relationship
g = Graph('neo4j://localhost:7687', auth = ('neo4j', 'ys1203303'))
g.run("MATCH (n) DETACH DELETE n") 
for i in range(999):
    node1 = Node(
            "parent",
            Name = newrules[i],
            Component = components[i],
            originalRule = rules[i],
            Type = "Derive",
            ProjectName = type_and_etc[i][0],
            Version = type_and_etc[i][1],
            owner = type_and_etc[i][3], 
            date = type_and_etc[i][2],
            ruleIndex = sections[i])
    g.create(node1) 
    section_df = df[df['Section_id'] == i + 1] 
    for s, ss in section_df.iterrows():
        node2 = Node(
            "son", 
            Name = ss['name'],
            Component = components[i],
            originalRule = rules[i],
            Type = "Derive",
            ProjectName = type_and_etc[i][0],
            Version = type_and_etc[i][1],
            owner = type_and_etc[i][3], 
            date = type_and_etc[i][2],
            ruleIndex = sections[i])
        g.create(node2)
        if ss['Status'] == '✔':
            relationship = Relationship(node2, "should", node1) 
        else:
            relationship = Relationship(node2, "should not", node1) 
        g.create(relationship) 